#### Import Packages

In [1]:
from IPython.display import display, clear_output
from tkinter import Tk, filedialog
from tqdm import tqdm
import pandas as pd
import numpy as np
import csv
import re

#### Import CSV file

In [2]:
# Create a Tkinter root window
root = Tk()
root.withdraw()  # Hide the root window

# Ask the user to select a CSV file
file_path = filedialog.askopenfilename(filetypes=[("CSV files", "*.csv")])

# Check if a file was selected
if file_path:
    # Load the CSV file into a pandas DataFrame
    df = pd.read_csv(file_path, encoding='latin-1')
    
    # Display the DataFrame
    print("CSV file loaded successfully. The shape of the data is " + str(df.shape))
    
else:
    print("No file selected.")

CSV file loaded successfully. The shape of the data is (20310, 41)


C:\Users\balaj\AppData\Local\Temp\ipykernel_18784\3719102924.py:11: DtypeWarning: Columns (19,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding='latin-1')


#### Duplicate the CSV for future use

In [3]:
df_dupe = df

In [4]:
df = df_dupe

#### Drops Rows where important values are missing

In [5]:
df = df.dropna(subset=['latest_version_git_repo'])

In [6]:
df = df.dropna(subset=['Latest Version'])

In [7]:
df.shape

(10628, 41)

#### Clean Github Links

Check if 'latest_version_git_repo' is a github link. If not, drop it. Also modifies @github.com: to 'https://github.com/'

In [8]:
def clean_github_links(df):
    link_change_count = 0
    row_drop_count = 0
    for i in tqdm(df.index):
        try:
            if (re.match(r'.*(github.com).*', df['latest_version_git_repo'][i])):
                if(re.match(r'.*(@github.com:).*', df['latest_version_git_repo'][i])):
                    df.at[i, 'latest_version_git_repo'] = re.sub(r'.*(@github.com:)', "https://github.com/", df['latest_version_git_repo'][i])
                    link_change_count += 1
            else:
                df = df.drop(index=i)
                row_drop_count += 1
        except:
            df = df.drop(index=i)
            row_drop_count += 1
    clear_output(wait=True)
    print("Github links modified: " + str(link_change_count))
    print("Rows dropped: ", str(row_drop_count))
    print("New data shape: ", str(df.shape))
    return df

In [9]:
df = clean_github_links(df)

Github links modified: 103
Rows dropped:  250
New data shape:  (10378, 41)


#### Clean Question Marks

Replacing '?' with __nan__ which is displayed as blank cell in csv

In [10]:
def remove_qmarks(val):
    if(val == '?'):
        return np.NaN
    else:
        return val

In [11]:
def clean_qmarks(column_name):
    df[column_name] = df[column_name].apply(remove_qmarks)
    return df[column_name]

In [12]:
for column in df.columns:
    df[column] = clean_qmarks(column)

#### Handle 'K' in Counts

In [13]:
def modify_K(val):
    if(re.match(r'([0-9.])+(K|k)', str(val))):
        val = str(round(float(val[:-1]) * 1000))
    return val

In [14]:
def clean_k_counts(column_name):
    df[column_name] = df[column_name].apply(modify_K)
    return df[column_name]

In [15]:
columns_to_handle = ['GitHub Stars', 'Forks', 'Versions', '# of Files']
for column in columns_to_handle:
    df[column] = clean_k_counts(column)

#### Clean timestamps

In [16]:
def change_timestamps(column_name):
    df[column_name] = pd.to_datetime(df[column_name])
    df[column_name] = df[column_name].dt.strftime('%Y-%m-%d')
    return df[column_name]

In [17]:
columns_to_handle = ['modified', 'created']
for column in columns_to_handle:
    df[column] = change_timestamps(column)

#### Clean "years ago" values

In [18]:
def extract_years(value):
    try:
        match = re.search(r'(\d+)\s*(years?|months?|days?|hours?)\s*(ago)*', value)
        if match:
            num_value = int(match.group(1)) 
            unit = match.group(2)
            if unit.startswith('year'):
                return round(num_value * 12, 2)
            elif unit.startswith('month'):
                return round(num_value, 2)
            elif unit.startswith('day'):
                return round(num_value / 30.44, 2)
            elif unit.startswith('hour'):
                return round(num_value / 720, 2)
        else:
            return np.NaN
    except:
        return value

In [19]:
def handle_years(column_name):
    df[column_name] = df[column_name].apply(extract_years)
    return df[column_name]

In [20]:
columns_to_handle = ['Latest Version', 'Last Commit', 'Age', 'Last Release']
for column in columns_to_handle:
    df[column] = handle_years(column)

#### Drop 'last release'. Redundant with 'Latest Version'

In [21]:
df = df.drop('Last Release', axis = 1)
df.shape

(10378, 40)

#### Drop 'Maintainers'. Redundant with 'maintainers_length'

In [22]:
count = 0
for i in tqdm(df.index):
    if not (df['maintainers_length'][i] == str(round(df['Maintainers'][i]))):
        count += 1
clear_output(wait=True)
count

5

In [23]:
df = df.drop('Maintainers', axis = 1)
df.shape

(10378, 39)

#### Clean '-' in 'Contributors' 

In [24]:
def remove_minus(val):
    if(val == '-'):
        return np.NaN
    else:
        return val

In [25]:
def clean_minus(column_name):
    df[column_name] = df[column_name].apply(remove_minus)
    return df[column_name]

In [26]:
columns_to_handle = ['Contributors']
for column in columns_to_handle:
    df[column] = clean_minus(column)

#### Handle 'Install Size' Column

In [27]:
def extract_size(value):
    try:
        match = re.search(r'([0-9.]+) *(B|kB|MB|GB)', str(value), re.IGNORECASE)
        if match:
            num_value = float(match.group(1)) 
            unit = match.group(2)
            if re.match(r'(B)', unit, re.IGNORECASE):
                return round(num_value / 1000, 2)
            elif re.match(r'(kB)', unit, re.IGNORECASE):
                return round(num_value, 2)
            elif re.match(r'(MB)', unit, re.IGNORECASE):
                return round(num_value * 1000, 2)
            elif re.match(r'(GB)', unit, re.IGNORECASE):
                return round(num_value * 1000000, 2)
        else:
            return np.NaN
    except:
        print(value)
        return value

In [28]:
def handle_size(column_name):
    df[column_name] = df[column_name].apply(extract_size)
    return df[column_name]

In [29]:
df['Install Size'] = handle_size('Install Size')

#### Change Column Type for Health Score

In [31]:
count = 0
for i in tqdm(df.index):
    try:
        if not (re.match(r'[0-9.]+', str(df['Health Score'][i]))):
            df.at[i, 'Health Score'] = np.nan
            count += 1
    except:
        print(df['Health Score'][i])
count

100%|████████████████████████████████████████████████████████████████████████| 10378/10378 [00:00<00:00, 155071.20it/s]


2

In [32]:
df['Health Score'] = df['Health Score'].astype('float')

#### Merge vulnerability columns

In [33]:
def merge_vul(columns_to_handle):
    for columns in columns_to_handle:
        df[columns] = df[columns].fillna(0)
    for i in tqdm(df.index):
        df.at[i, columns_to_handle[0]] = df.at[i, columns_to_handle[0]] + df.at[i, columns_to_handle[1]] + df.at[i, columns_to_handle[2]] + df.at[i, columns_to_handle[3]]
        df.at[i, columns_to_handle[0]] = int(round(df.at[i, columns_to_handle[0]]))
    return df

In [34]:
columns_to_handle = ['critical_vul_count', 'high_vul_count', 'medium_vul_count', 'low_vul_count']
df = merge_vul(columns_to_handle)
df = df.rename(columns={columns_to_handle[0]: 'vul_count'})
df = df.drop(columns_to_handle[1:4], axis = 1)

100%|█████████████████████████████████████████████████████████████████████████| 10378/10378 [00:00<00:00, 29944.80it/s]


#### Changing 'deprecation_status' values to bool

In [35]:
def modify_bool(val):
    if(val == 'Yes'):
        return True
    elif(val == 'No'):
        return False
    else:
        return np.nan

In [36]:
def clean_depricated(column_name):
    df[column_name] = df[column_name].apply(modify_bool)
    return df[column_name]

In [37]:
df['deprecated'] = clean_depricated('deprecated')

#### Renaming Columns

In [38]:
old_col_names = ['created', 'modified', 'latest_version_git_repo', 'deprecated', 'Latest Version', 'Forks', 'Contributors', 'readmeFilename_exists', 'Readme File', 'Age', 'Dependencies', 'Versions', 'Last Commit']
new_col_names = ['package_created', 'last_modified', 'git_repository', 'deprecation_status', 'months_since_latest_version_synk', 'github_forks', 'github_contributors', 'readme_exists_npmjs', 'readme_exists_synk', 'package_age', 'dependencies_synk', 'num_versions_released', 'months_since_last_commit']
for i in range(len(old_col_names)):
    df = df.rename(columns={old_col_names[i]: new_col_names[i]})
    print("\'" + old_col_names[i] + "\' renamed to \'" + new_col_names[i] + "\'")

'created' renamed to 'package_created'
'modified' renamed to 'last_modified'
'latest_version_git_repo' renamed to 'git_repository'
'deprecated' renamed to 'deprecation_status'
'Latest Version' renamed to 'months_since_latest_version_synk'
'Forks' renamed to 'github_forks'
'Contributors' renamed to 'github_contributors'
'readmeFilename_exists' renamed to 'readme_exists_npmjs'
'Readme File' renamed to 'readme_exists_synk'
'Age' renamed to 'package_age'
'Dependencies' renamed to 'dependencies_synk'
'Versions' renamed to 'num_versions_released'
'Last Commit' renamed to 'months_since_last_commit'


#### Changing Column Types

In [61]:
for column in df.columns:
    print(str(df[column].dtype) + "\t\t:\t" + column)

object		:	package_name
object		:	description
int32		:	maintainers_length
bool		:	readme_exists_npmjs
datetime64[ns]		:	package_created
datetime64[ns]		:	last_modified
object		:	latest_version
object		:	git_repository
object		:	dependencies_name
int32		:	dependencies_count
object		:	dev_dependencies_name
int32		:	dev_dependencies_count
bool		:	deprecation_status
float64		:	last_day_downloads
float64		:	last_week_downloads
float64		:	last_month_downloads
float64		:	last_3_month_downloads
float64		:	months_since_latest_version_synk
object		:	License
float64		:	Health Score
object		:	Security Recommendation
object		:	Popularity
object		:	Maintainence
object		:	Community
float64		:	GitHub Stars
float64		:	github_forks
float64		:	github_contributors
float64		:	months_since_last_commit
object		:	readme_exists_synk
int64		:	package_age
object		:	dependencies_synk
int64		:	num_versions_released
float64		:	Install Size
int64		:	# of Files
object		:	TS Typings
float64		:	vul_count


In [40]:
df['package_name'] = df['package_name'].astype('str')

In [41]:
df['description'] = df['description'].astype('str')

In [42]:
df['maintainers_length'] = df['maintainers_length'].astype('int32')

In [43]:
df['readme_exists_npmjs'] = df['readme_exists_npmjs'].astype('bool')

In [44]:
df['package_created'] = df['package_created'].astype('datetime64')

In [45]:
df['last_modified'] = df['last_modified'].astype('datetime64')

In [46]:
df['latest_version'] = df['latest_version'].astype('str')

In [47]:
df['git_repository'] = df['git_repository'].astype('str')

In [48]:
df['dependencies_count'] = df['dependencies_count'].astype('int32')

In [49]:
df['dev_dependencies_count'] = df['dev_dependencies_count'].astype('int32')

In [50]:
df['deprecation_status'] = df['deprecation_status'].astype('bool')

In [51]:
df['License'] = df['License'].astype('str')

In [52]:
df['Security Recommendation'] = df['Security Recommendation'].astype('str')

In [53]:
df['Popularity'] = df['Popularity'].astype('str')

In [54]:
df['Maintainence'] = df['Maintainence'].astype('str')

In [55]:
df['Community'] = df['Community'].astype('str')

In [56]:
df['GitHub Stars'] = df['GitHub Stars'].astype('float64')

In [57]:
df['github_forks'] = df['github_forks'].astype('float64')

In [58]:
df['github_contributors'] = df['github_contributors'].astype('float64')

In [59]:
df['num_versions_released'] = df['num_versions_released'].astype('int64')

In [60]:
df['# of Files'] = df['# of Files'].astype('int64')

#### Check Value Freq for a Column

In [62]:
column_name = "deprecation_status"
print(df[column_name].value_counts(dropna = False))

False    9975
True      403
Name: deprecation_status, dtype: int64


#### Save Cleaned Data to CSV

In [63]:
df.to_csv('cleaned_data.csv', index=False)

In [64]:
df.shape

(10378, 36)